## Import

In [1]:
import random
import pandas as pd
import numpy as np
import os
from tqdm.auto import tqdm
import librosa

from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import warnings
warnings.filterwarnings(action='ignore')

## Hyperparameter Setting

In [63]:
CFG = {
    'SR':16000,
    'N_MFCC':128, # Melspectrogram 벡터를 추출할 개수
    'SEED':42
}

## Fixed Random-Seed

In [64]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(CFG['SEED']) # Seed 고정

## Data Pre-Processing

In [65]:
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

In [78]:
def get_mfcc_feature(df):
    features = []
    for path in tqdm(df['path']):
        y, sr = librosa.load(path, sr=CFG['SR'])
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=CFG['N_MFCC'])
        features.append({
            'mfcc_mean': np.mean(mfcc, axis=1),
            'mfcc_max': np.max(mfcc, axis=1),
            'mfcc_min': np.min(mfcc, axis=1),
        })

    mfcc_df = pd.DataFrame(features)
    mfcc_mean_df = pd.DataFrame(mfcc_df['mfcc_mean'].tolist(), columns=[f'mfcc_mean_{i}' for i in range(CFG['N_MFCC'])])
    mfcc_max_df = pd.DataFrame(mfcc_df['mfcc_max'].tolist(), columns=[f'mfcc_max_{i}' for i in range(CFG['N_MFCC'])])
    mfcc_min_df = pd.DataFrame(mfcc_df['mfcc_min'].tolist(), columns=[f'mfcc_min_{i}' for i in range(CFG['N_MFCC'])])

    return pd.concat([mfcc_mean_df, mfcc_max_df, mfcc_min_df], axis=1)


def get_feature_mel(df):
    features = []
    for path in tqdm(df['path']):
        data, sr = librosa.load(path, sr=CFG['SR'])
        n_fft = 2048
        win_length = 2048
        hop_length = 1024
        n_mels = 128

        D = np.abs(librosa.stft(data, n_fft=n_fft, win_length = win_length, hop_length=hop_length))
        mel = librosa.feature.melspectrogram(S=D, sr=sr, n_mels=n_mels, hop_length=hop_length, win_length=win_length)

        features.append({
            'mel_mean': mel.mean(axis=1),
            'mel_max': mel.min(axis=1),
            'mel_min': mel.max(axis=1),
        })
    mel_df = pd.DataFrame(features)
    mel_mean_df = pd.DataFrame(mel_df['mel_mean'].tolist(), columns=[f'mel_mean_{i}' for i in range(n_mels)])
    mel_max_df = pd.DataFrame(mel_df['mel_max'].tolist(), columns=[f'mel_max_{i}' for i in range(n_mels)])
    mel_min_df = pd.DataFrame(mel_df['mel_min'].tolist(), columns=[f'mel_min_{i}' for i in range(n_mels)])

    return pd.concat([mel_mean_df, mel_max_df, mel_min_df], axis=1)

In [79]:
train_mf = get_mfcc_feature(train_df)
test_mf = get_mfcc_feature(test_df)

  0%|          | 0/5001 [00:00<?, ?it/s]


KeyboardInterrupt



In [80]:
train_mel = get_feature_mel(train_df)
test_mel = get_feature_mel(test_df)

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/1881 [00:00<?, ?it/s]

In [81]:
train_x = pd.concat([train_mel, train_mf], axis=1)
test_x = pd.concat([test_mel, test_mf], axis=1)

In [82]:
train_y = train_df['label']

In [83]:
train_x['label'] = train_df['label']
train_data = TabularDataset(train_x)
test_data = TabularDataset(test_x)

In [84]:
train_x

,mel_mean_0,mel_mean_1,mel_mean_2,mel_mean_3,mel_mean_4,mel_mean_5,mel_mean_6,mel_mean_7,mel_mean_8,mel_mean_9,...,mfcc_min_119,mfcc_min_120,mfcc_min_121,mfcc_min_122,mfcc_min_123,mfcc_min_124,mfcc_min_125,mfcc_min_126,mfcc_min_127,label
0,0.084566,0.056834,0.068925,0.102323,0.096968,0.086162,0.094872,0.122292,0.132902,0.129273,...,-3.878134,-4.017255,-4.249367,-3.223005,-4.603191,-1.934261,-3.485538,-2.463422,-3.229240,1
1,0.098563,0.052329,0.056681,0.061517,0.081819,0.106868,0.138239,0.107013,0.144104,0.120996,...,-3.810820,-4.056773,-4.705102,-4.370793,-2.665026,-5.128587,-3.855331,-3.354254,-3.015537,2
2,0.083089,0.055145,0.060235,0.078781,0.078231,0.141836,0.142165,0.156169,0.129583,0.136608,...,-4.208199,-4.048993,-3.270407,-3.567140,-5.038786,-3.207233,-5.000182,-4.171637,-3.029602,4
3,0.020401,0.055993,0.088625,0.112177,0.166121,0.092082,0.082991,0.106938,0.118478,0.130794,...,-4.395284,-2.994612,-4.725981,-3.209173,-4.101067,-5.370552,-3.596650,-4.457966,-4.496696,5
4,0.074340,0.047811,0.056292,0.110727,0.126938,0.169690,0.195548,0.141796,0.155485,0.272525,...,-2.485117,-3.151918,-4.735774,-3.539434,-3.760095,-3.584368,-3.163038,-4.498411,-4.766491,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4996,0.021166,0.058369,0.111723,0.110858,0.119897,0.068029,0.117166,0.131592,0.119517,0.130425,...,-2.708395,-4.064007,-4.244311,-3.986595,-3.176047,-3.143501,-4.341457,-2.936266,-3.653713,5
4997,0.094659,0.061243,0.070688,0.099395,0.105742,0.081150,0.080464,0.094489,0.137502,0.163564,...,-3.696560,-3.079943,-6.046649,-5.025594,-4.648950,-2.863224,-3.787347,-3.878424,-3.689407,0
4998,0.120308,0.064221,0.062565,0.062162,0.088277,0.158967,0.121889,0.090846,0.138761,0.148893,...,-2.578661,-4.233521,-4.165043,-3.027785,-5.933722,-2.864382,-4.454404,-3.263407,-4.237404,1
4999,0.074006,0.055329,0.062009,0.098595,0.098970,0.073677,0.073534,0.102842,0.120538,0.151261,...,-4.765524,-5.089233,-4.338481,-4.685366,-4.513367,-5.112647,-3.516397,-3.337688,-4.203060,1


# autogluon

In [85]:
label = 'label'
eval_metric = 'accuracy'
time_limit = 3600 * 24

In [86]:
predictor = TabularPredictor(
    label=label, eval_metric=eval_metric
).fit(train_data, presets='best_quality', time_limit=time_limit, ag_args_fit={'num_gpus': 1, 'num_cpus': 32})

No path specified. Models will be saved in: "AutogluonModels/ag-20230509_081645/"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 86400s
AutoGluon will save models to "AutogluonModels/ag-20230509_081645/"
AutoGluon Version:  0.7.0
Python Version:     3.8.10
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #74-Ubuntu SMP Wed Feb 22 14:14:39 UTC 2023
Train Data Rows:    5001
Train Data Columns: 768
Label Column: label
Preprocessing data ...
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == int, but few unique label-values observed).
	6 unique label values:  [1, 2, 4, 5, 3, 0]
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Train Data Class Count:

In [87]:
predictor.leaderboard(silent=True)

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,0.604679,76.172411,31351.237048,0.001259,1.139999,3,True,26
1,CatBoost_BAG_L2,0.603479,69.998960,26563.986655,4.751987,1114.998491,2,True,20
2,WeightedEnsemble_L2,0.601480,47.154984,12412.085151,0.001246,1.328684,2,True,14
3,LightGBMXT_BAG_L2,0.595081,66.354195,27638.816787,1.107221,2189.828623,2,True,16
4,LightGBM_BAG_L2,0.593081,66.492281,28411.932770,1.245307,2962.944606,2,True,17
5,XGBoost_BAG_L2,0.592082,69.195677,27270.558099,3.948703,1821.569935,2,True,23
6,LightGBMLarge_BAG_L2,0.587882,67.738774,34563.913206,2.491800,9114.925042,2,True,25
7,NeuralNetFastAI_BAG_L1,0.586483,3.351180,1659.003203,3.351180,1659.003203,1,True,3
8,ExtraTreesEntr_BAG_L2,0.579684,66.225155,25450.584017,0.978181,1.595853,2,True,22
9,RandomForestGini_BAG_L2,0.577884,65.872273,25451.116867,0.625299,2.128704,2,True,18


## Inference

In [88]:
model_to_use = predictor.get_model_best()
model_pred = predictor.predict(test_data, model=model_to_use)

## Submission

In [89]:
submission = pd.read_csv('./sample_submission.csv')
submission['label'] = model_pred
submission.to_csv('./autogluon_음성감정.csv', index=False)